In [3]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector
import numpy as np

# Define the QUBO function
def qubo_cost(x):
    return x[0] + x[1] + 2 * x[0] * x[1]

# Create oracle that marks all bitstrings with cost < threshold
def create_threshold_oracle(n, threshold, qubo_fn):
    qc = QuantumCircuit(n)
    for i in range(2**n):
        bits = format(i, f"0{n}b")
        x = [int(b) for b in bits]
        if qubo_fn(x) < threshold:
            for j, b in enumerate(bits):
                if b == '0':
                    qc.x(j)
            qc.h(n-1)
            qc.mcx(list(range(n-1)), n-1)
            qc.h(n-1)
            for j, b in enumerate(bits):
                if b == '0':
                    qc.x(j)
    return qc

# Diffuser
def grover_diffuser(n):
    qc = QuantumCircuit(n)
    qc.h(range(n))
    qc.x(range(n))
    qc.h(n-1)
    qc.mcx(list(range(n-1)), n-1)
    qc.h(n-1)
    qc.x(range(n))
    qc.h(range(n))
    return qc

# Run GAS
n = 2
num_iterations = 3
threshold = 999
best_solution = None

for _ in range(num_iterations):
    # Generate all possible bitstrings and evaluate cost
    all_states = [format(i, '0' + str(n) + 'b') for i in range(2**n)]
    costs = [qubo_cost([int(b) for b in state]) for state in all_states]
    
    # Find current best
    if best_solution is None:
        threshold = min(costs)
    else:
        threshold = min(threshold, qubo_cost(best_solution))

    oracle = create_threshold_oracle(n, threshold, qubo_cost)
    diffuser = grover_diffuser(n)

    qc = QuantumCircuit(n)
    qc.h(range(n))
    qc.append(oracle, range(n))
    qc.append(diffuser, range(n))

    sv = Statevector.from_instruction(qc)
    probs = sv.probabilities_dict()

    # Take the most probable bitstring
    best_candidate = max(probs, key=probs.get)
    candidate_bits = [int(b) for b in best_candidate]

    if best_solution is None or qubo_cost(candidate_bits) < qubo_cost(best_solution):
        best_solution = candidate_bits

print("Best solution found:", best_solution)
print("Minimum cost:", qubo_cost(best_solution))


Best solution found: [0, 0]
Minimum cost: 0
